# Import packages

In [ ]:
# !pip install -r multiff_analysis/requirements.txt

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys

project_folder = '/Users/dusiyi/Documents/Multifirefly-Project'
os.chdir(project_folder)
sys.path.append(os.path.join(project_folder, 'multiff_analysis', 'methods'))

from non_behavioral_analysis.neural_data_analysis.model_neural_data import transform_vars, neural_data_modeling
from non_behavioral_analysis.neural_data_analysis.model_neural_data.cca_methods import cca_class, cca_utils, cca_lag_vs_no_lag_plotting, cca_cv_utils, cca_cv_plotting, cca_plotting
from non_behavioral_analysis.neural_data_analysis.decode_targets import decode_target_class, prep_decode_target
from data_wrangling import specific_utils, base_processing_class, general_utils

import sys

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc

# Machine Learning imports
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from statsmodels.multivariate.cancorr import CanCorr

# Neuroscience specific imports
import rcca

# To fit gpfa
import numpy as np
import neo
import matplotlib.pyplot as plt
from importlib import reload

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
print("done")


pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)


%load_ext autoreload
%autoreload 2

# Get data

In [ ]:
reload(decode_target_class)

In [ ]:
#raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0416"
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0328"
dec = decode_target_class.DecodeTargetClass(raw_data_folder_path=raw_data_folder_path,
                                                               bin_width=0.02, window_width=0.05)
dec.get_x_and_y_data_for_modeling(exists_ok=True)

In [ ]:
cca_no_lag = cca_class.CCAclass(X1=dec.x_var_reduced, X2=dec.y_var_reduced, lagging_included=False)
cca_no_lag.conduct_cca()

cca_lags = cca_class.CCAclass(X1=dec.x_var_lags_reduced.drop(columns='bin'), X2=dec.y_var_lags_reduced, lagging_included=True)
# for all columns that end with _0, rename them to the column name without the _0
cca_lags.X2.columns = cca_lags.X2.columns.str.replace('_0', '')
cca_lags.conduct_cca()


print(f'dec.x_var_lags.shape: {dec.x_var_lags.shape}')
print(f'dec.y_var_lags_reduced.shape: {dec.y_var_lags_reduced.shape}')

cca_inst = cca_lags

# CCA

https://medium.com/@pozdrawiamzuzanna/canonical-correlation-analysis-simple-explanation-and-python-example-a5b8e97648d2

## compare lag vs no lag

In [17]:
canon_df = pd.DataFrame(cca_no_lag.canon_corr, columns = ['no_lag'])
canon_df[f'with_lags'] = cca_lags.canon_corr
canon_df['component'] = [f'CC {i+1}' for i in range(cca_lags.n_components)]
# convert canon_df to long format
canon_df_long = pd.melt(canon_df, id_vars=['component'], var_name='lag', value_name='canon_coeff')

In [ ]:
# make a sns bar plot on canon_df_long
plt.figure(figsize=(8, 6))
sns.barplot(x='component', y='canon_coeff', data=canon_df_long, hue='lag')
plt.show()

## cca_inst (choose one between lags and no lag)

In [19]:
# choose no lag
cca_inst = cca_inst

In [20]:
# choose lags
cca_inst = cca_lags

# Barplots of loadings/weights

## loadings

### neurons

In [ ]:
cca_inst.plot_ranked_loadings(X1_or_X2='X1', squared=False)

### behavior

In [ ]:
cca_inst.plot_ranked_loadings(X1_or_X2='X2', squared=False)

## squared loadings

### neurons

In [ ]:
cca_inst.plot_ranked_loadings(X1_or_X2='X1')

### behavior

In [ ]:
cca_inst.plot_ranked_loadings(X1_or_X2='X2')

## weights (ranked)

### neurons

In [ ]:
cca_inst.plot_ranked_weights(abs_value=False)

### behavior

In [ ]:
cca_inst.plot_ranked_weights(X1_or_X2='X2', abs_value=False)

## abs weights ranked

### neurons

In [ ]:
cca_inst.plot_ranked_weights()

### behavior

In [ ]:
cca_inst.plot_ranked_weights(X1_or_X2='X2')

In [ ]:
stop here!

## distribution of each feature

In [ ]:
cca_inst.X2_sc.shape

In [ ]:
X2_sc_df = pd.DataFrame(cca_inst.X2_sc, columns = cca_inst.X2.columns)
X2_sc_df.describe()

In [ ]:
max_plot_to_make = 3

counter = 0
for column in X2_sc_df.columns:
    counter += 1
    if counter > max_plot_to_make:
        break
    
    plt.figure(figsize=(8, 2))
    sns.boxplot(X2_sc_df[column], orient='h')
    plt.show()
    

## heatmap of weights
Note: raw canonical coefficients are interpreted in a manner analogous to interpreting regression coefficients. For example: a one unit increase in reading leads to a .0446 decrease in the first canonical variate of set 2 when all of the other variables are held constant (in some other data)

In [56]:
weight_df = cca_inst.X2_weight_df.copy()
weight_df = weight_df.set_index('feature').drop(columns='feature_category')

In [ ]:
plt.subplots(figsize=(15, 25))
sns.heatmap(weight_df.iloc[:20, :10], cmap='coolwarm', annot=True, linewidths=1)
plt.show()

# Test for p values

In [ ]:
stats_cca = CanCorr(cca_inst.X1_sc, cca_inst.X2_sc)
print(stats_cca.corr_test().summary())
neural_data_modeling.print_weights('X', stats_cca.x_cancoef)
neural_data_modeling.print_weights('Z', stats_cca.y_cancoef)

## compute explained variance

In [ ]:
cca2.compute_ev([test1, test2])

In [ ]:
stop pls

# Train test split

## no lag: train vs test

In [ ]:
n_splits = 5
reg = 0.1
n_components = 10

cca_inst = cca_lags

X1_df = cca_inst.X1_sc_df
X2_df = cca_inst.X2_sc_df

# Canonical correlations
canon_corr_stats, cross_view_corr_stats = cca_cv_utils.crossvalidated_cca_analysis(X1_df, X2_df, n_components=n_components, reg=reg, n_splits=n_splits)


In [ ]:
cca_no_lag.traincorrs = cca2.validate([train1, train2])
cca_no_lag.testcorrs = cca2.validate([test1, test2])


In [ ]:
train1, test1, train2, test2 = train_test_split(cca_inst.X1_sc, cca_inst.X2_sc, test_size=0.3, random_state=42)
# use training and testing set
nComponents = 10
cca2 = rcca.CCA(kernelcca = False, reg = 0.1, numCC = nComponents)
cca2.train([train1, train2])

cca_inst.traincorrs = cca2.validate([train1, train2])
cca_inst.testcorrs = cca2.validate([test1, test2])

cca_plotting.plot_cca_prediction_accuracy_train_test_bars(cca_inst.traincorrs, cca_inst.testcorrs)
cca_plotting.plot_cca_prediction_accuracy_train_test_stacked_bars(cca_inst.traincorrs, cca_inst.testcorrs)
cca_plotting.plot_cca_prediction_accuracy_test_w_bars(cca_inst.traincorrs)
cca_plotting.plot_cca_prediction_accuracy_w_scatter(cca_inst.testcorrs)

## with lags: train vs test

In [107]:
# # check for problems in the data
# cca_lags.x_var_reduced = prep_decode_target.remove_zero_var_cols(
#             cca_lags.X2_sc_df)
# _ = prep_decode_target.remove_zero_var_cols(
#             test1)

In [ ]:

train1, test1, train2, test2 = train_test_split(cca_no_lag.X1_sc, cca_no_lag.X2_sc, test_size=0.3, random_state=42)
# use training and testing set
nComponents = 10
cca2 = rcca.CCA(kernelcca = False, reg=1e-4, numCC = nComponents)
cca2.train([train1, train2])

cca_no_lag.traincorrs = cca2.validate([train1, train2])
cca_no_lag.testcorrs = cca2.validate([test1, test2])



train1, test1, train2, test2 = train_test_split(cca_lags.X1_sc, cca_lags.X2_sc, test_size=0.3, random_state=42)
# use training and testing set
nComponents = 10
cca2 = rcca.CCA(kernelcca = False, reg = 0., numCC = nComponents)
cca2.train([train1, train2])

cca_lags.traincorrs = cca2.validate([train1, train2])
cca_lags.testcorrs = cca2.validate([test1, test2])


cca_plotting.plot_cca_prediction_accuracy_train_test_bars(cca_lags.traincorrs, cca_lags.testcorrs)
cca_plotting.plot_cca_prediction_accuracy_train_test_stacked_bars(cca_lags.traincorrs, cca_lags.testcorrs)
cca_plotting.plot_cca_prediction_accuracy_test_w_bars(cca_lags.traincorrs)
cca_plotting.plot_cca_prediction_accuracy_w_scatter(cca_lags.testcorrs)

## compare lags vs no lag

In [ ]:
cca_lag_vs_no_lag_plotting.plot_cca_prediction_accuracy_train_test_bars_for_lags_and_no_lags(cca_lags.traincorrs, cca_lags.testcorrs, cca_no_lag.traincorrs, cca_no_lag.testcorrs)

# exp

In [ ]:
n_splits = 5
reg = 0.1
n_components = 10

cca_inst = cca_lags

X1_df = cca_inst.X1_sc_df
X2_df = cca_inst.X2_sc_df

# Canonical correlations
canon_corr_stats, cross_view_corr_stats = cca_cv_utils.crossvalidated_cca_analysis(X1_df, X2_df, n_components=n_components, reg=reg, n_splits=n_splits)


In [ ]:
cca_lag_vs_no_lag_plotting.plot_cca_lag_vs_nolag_and_train_vs_test(combined_X2_df, 'DatasetName', mode='train_offset')


## compare lags vs no lag & train vs test

In [64]:
combined_X1_df, combined_X2_df = cca_lag_vs_no_lag_plotting.combine_data_to_compare_train_and_test(cca_no_lag, cca_lags)

In [ ]:
# can make this into a df based on combined_X2_df

cross_view_corr_stats.keys()

In [ ]:
combined_X2_df

In [ ]:
cca_lag_vs_no_lag_plotting.plot_cca_lag_vs_nolag_and_train_vs_test(combined_X2_df, 'DatasetName', mode='lag_offset')


In [ ]:
cca_lag_vs_no_lag_plotting.plot_cca_lag_vs_nolag_and_train_vs_test(combined_X2_df, 'DatasetName', mode='train_offset')


# Transform vars
e.g. use basis functions

In [ ]:
cca_no_lag.X2_tf_df = transform_vars.transform_behav_data(cca_no_lag.X2)

cca_lags.X2_tf_df = transform_vars.transform_behav_data(cca_lags.X2)



In [85]:
cca_inst = cca_lags

X1_df = cca_inst.X1_sc_df
X2_df = cca_inst.X2_tf_df


## run k-fold CV

In [ ]:
n_splits = 5
reg = 0.1
n_components = 10

# Canonical correlations
canon_corr_stats, cross_view_corr_stats = cca_cv_utils.crossvalidated_cca_analysis(X1_df, X2_df, n_components=n_components, reg=reg, n_splits=n_splits)


## plot

### cross view

In [ ]:
use_cross_view_corr = True
filter_significant = True
sort_by_significance = True
significance_threshold = 4

# Full plot
cca_cv_plotting.plot_cca_cv_results(cross_view_corr_stats, data_type='X1', component=0, use_cross_view_corr=use_cross_view_corr,
                                    filter_significant=filter_significant, sort_by_significance=sort_by_significance, significance_threshold=significance_threshold)
# Canonical correlation view
cca_cv_plotting.plot_cca_cv_results(cross_view_corr_stats, data_type='X2', component=0, use_cross_view_corr=use_cross_view_corr,
                                    filter_significant=filter_significant, sort_by_significance=sort_by_significance, significance_threshold=significance_threshold)


### canon corr

In [ ]:
use_cross_view_corr = False
filter_significant = True
sort_by_significance = True
significance_threshold = 4

# Full plot
cca_cv_plotting.plot_cca_cv_results(canon_corr_stats, data_type='X1', component=0, use_cross_view_corr=use_cross_view_corr,
                                    filter_significant=filter_significant, sort_by_significance=sort_by_significance, significance_threshold=significance_threshold)
# Canonical correlation view
cca_cv_plotting.plot_cca_cv_results(canon_corr_stats, data_type='X2', component=0, use_cross_view_corr=use_cross_view_corr,
                                    filter_significant=filter_significant, sort_by_significance=sort_by_significance, significance_threshold=significance_threshold)


# Appendix

## thoughts: could you use neural data to decode target position?

what about 2nd target's position?
(can either use 1st target's decoder, or train and separate decoder for 2nd target)

also...try GPFA at some point 

## other potential transformers to use

In [ ]:
transformers = [
    ('identity', lambda x: x),
    ('log_signed', lambda x: np.log1p(np.abs(x)) * np.sign(x)),
    ('poly', lambda x: np.hstack([x**p for p in [1, 2, 3]])),
    ('sqrt_signed', lambda x: np.sqrt(np.abs(x)) * np.sign(x)),
    ('rank_gauss', rank_gauss_transform)  # optional custom
]


## check NA

In [ ]:
general_utils.find_rows_with_na(cca_inst.X2_sc_df)

In [ ]:
general_utils.check_array_integrity(X_t, name="Array", top_n=10, verbose=True)

## permutation test (later)

In [ ]:
import numpy as np
from tqdm import tqdm

n_permutations = 1000
component = 0  # which canonical component
n_features = X1.shape[1]

# Store observed loadings
observed_loadings = mean_X1_test[:, component]  # or use your preferred estimate

# Store null distributions
null_loadings = np.zeros((n_features, n_permutations))

for perm in tqdm(range(n_permutations)):
    # Shuffle rows of X1
    X1_perm = np.random.permutation(X1)
    # Fit CCA on permuted X1 and original X2
    cca2 = rcca.CCA(kernelcca=False, reg=0.1, numCC=nComponents)
    cca2.train([X1_perm, X2])
    U_perm = np.dot(X1_perm, cca2.ws[0])
    # Compute loadings for permuted data
    load_X1_perm = np.corrcoef(X1_perm.T, U_perm.T)[:n_features, n_features:]
    null_loadings[:, perm] = load_X1_perm[:, component]

# Compute two-sided p-values
p_values = np.mean(np.abs(null_loadings) >= np.abs(observed_loadings[:, None]), axis=1)

In [ ]:
import matplotlib.pyplot as plt

alpha = 0.05  # significance level
significant_mask = p_values < alpha

sig_means = observed_loadings[significant_mask]
sig_stds = std_X1_test[significant_mask, component]  # or use std from CV
sig_labels = np.array(x1_labels)[significant_mask]

plt.figure(figsize=(12, 5))
plt.bar(sig_labels, sig_means, yerr=sig_stds, capsize=5, color='skyblue')
plt.xticks(rotation=90)
plt.ylabel('Mean Canonical Loading')
plt.title(f'Significant Canonical Loadings (p < {alpha})')
plt.tight_layout()
plt.show()